# Принцип работы

## Решение основано на BERT.
1. На вход для обучения подаётся файл по пути из **train_path**
2. Из текста удаляются знаки препинания
3. Затем текст и ответы переводятся в тензоры
4. Всё загружается в датасет и делится на две части в соотношении 9 к 1
5. Подгружается модель, и настраиваются параметры обучения
6. Происходит обучение модели (требуется мощная видеокарта)
7. Загружается тестовый датасет по пути из **test_path**, в котором также чистится текст, а затем переводится в тензоры и загружаются в датасет
8. Данные подаются модели и полученный ответ обрабатывается и сохраняется в новый столбец датафрейма
9. Столбец с текстом удаляется и полученный датафрейм экспортируется по пути из **ans_path**

In [ ]:
!pip install transformers

**Установка библиотек**

Обучение происходило на kaggle. Если на устройстве нет ниже указанных библиотек, то запустите их установку.

In [ ]:
!pip install -U scikit-learn
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install pandas
!pip install numpy

In [ ]:
import time
import datetime

import transformers
from transformers import AutoTokenizer, AutoModel
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import torch
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
import pandas as pd
import numpy as np

In [ ]:
train_path = "/kaggle/input/sirius-task-1/train.csv"
test_path = "/kaggle/input/sirius-task-1/test.csv"
ans_path = '/kaggle/working/solution.csv'

In [ ]:
df = pd.read_csv(train_path)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
df['text'] = df['text'].replace("[0-9!#()$\,\'\-\.*+/:;<=>?@[\]^_`{|}\"]+", ' ', regex=True)
df['text'] = df['text'].replace(r'\s+', ' ', regex=True)

,ID,text,sentiment
0,21098,с и спросил его о Посланник Аллаха Ты порицае...,1
1,21099,Роднее всех родных Попала я в ГКБ № еще в дека...,1
2,21100,Непорядочное отношение к своим работникам Рабо...,2
3,21101,Отсутствуют нормативы Госты и прочее что позв...,1
4,21102,У меня машина в руках лет и это первая моя ма...,1
5,21103,Самое лучшее в отеле место его расположения в ...,0
6,21104,Дорогая аптека Дорогая аптека В других сетях ц...,1
7,21105,Останавливались с женой на годовщину Отель пол...,1
8,21106,Персонал вежливый все сделали аккуратно В цело...,1
9,21107,Второй раз останавливаюсь в этом отеле С первы...,0


In [ ]:
text = df['text'].values
sentiment = df['sentiment'].values

In [ ]:
input_ids = []
attention_masks = []

for sent in text:
    encoded_dict = tokenizer.encode_plus(sent,
                                         add_special_tokens=True,
                                         max_length=128,
                                         truncation=True,
                                         pad_to_max_length=True,
                                         return_attention_mask=True,
                                         return_tensors='pt',
                                         )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
sentiment = torch.tensor(sentiment)

In [ ]:
dataset = TensorDataset(input_ids, attention_masks, sentiment)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

170901 18990 <class 'torch.utils.data.dataset.Subset'>


In [ ]:
batch_size = 32

train_dataloader = DataLoader(train_dataset,
                              sampler=RandomSampler(train_dataset),
                              batch_size=batch_size)

validation_dataloader = DataLoader(val_dataset,
                                   sampler=SequentialSampler(val_dataset),
                                   batch_size=batch_size)

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased",
    num_labels=3,
    output_attentions=False,
    output_hidden_states = False
)

model.cuda()

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
epochs = 2

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps)

In [ ]:
def flat_f1(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
def format_time(elapsed):
    elapsed_rounded = int(round(elapsed))

    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
device = torch.device('cuda')

In [ ]:
import random

#для воспроизводимости результата
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []

total_t0 = time.time()

for epoch in range(0, epochs):
    print("")
    print(f"=====Эпоха {epoch+1} из {epochs}")
    print("Идёт обучение...")

    t0 = time.time()
    total_train_loss = 0

    # размарозка весов модели для обучения
    model.train()

    for step, batch in enumerate(train_dataloader):
        if step%500 == 0 and step != 0:
            elapsed = format_time(time.time() - t0)
            print(f" Batch {step} of {len(train_dataloader)} Время: {elapsed}")

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_sentiment = batch[2].to(device)

        model.zero_grad()
        #обнуление градиентов

        res = model(b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_sentiment)
        loss = res['loss']
        logits = res['logits']

        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)
    training_time = format_time(time.time() - t0)
    print("")
    print(f" Loss:{avg_train_loss}")
    print(f" Время тренировки {training_time}")

    print("")
    print("Идёт валидация")

    t0 = time.time()
    model.eval()
    # замораживаем веса

    total_eval_f1 = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:


        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_sentiment = batch[2].to(device)

        with torch.no_grad():
            res = model(b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_sentiment)
            loss = res['loss']
            logits = res['logits']

        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        sentiments_ids = b_sentiment.to('cpu').numpy()

        total_eval_f1 += flat_f1(logits, sentiments_ids)

    avg_val_f1 = total_eval_f1 / len(validation_dataloader)

    print(f" Валидация F1: {avg_val_acc}")

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)

    print(f" Валидационная ошибка: {avg_val_loss}")
    print(f" Время валидации {validation_time}")

    training_stats.append(
        {"epoch": epoch+1,
         'train_loss': avg_train_loss,
         'valid_loss': avg_val_loss,
         'valid_acc': avg_val_f1,
         'train_time': training_time,
         'valid_time': validation_time})

print("")
print("Обучение закончено")
print(f"Общее время обучения: {time.time()-total_t0}")


=====Эпоха 1 из 2
Идёт обучение...
 Batch 500 of 5341 Время: 0:05:38
 Batch 1000 of 5341 Время: 0:11:25
 Batch 1500 of 5341 Время: 0:17:13
 Batch 2000 of 5341 Время: 0:23:00


KeyboardInterrupt: 

In [ ]:
data = pd.read_csv(test_path)
data['text'] = data['text'].replace("[0-9!#()$\,\'\-\.*+/:;<=>?@[\]^_`{|}\"]+", ' ', regex=True)
data['text'] = data['text'].replace(r'\s+', ' ', regex=True)

text = data['text'].values

input_ids = []
attention_masks = []

for sent in text:
    encoded_dict = tokenizer.encode_plus(sent,
                                         add_special_tokens=True,
                                         max_length=128,
                                         truncation=True,
                                         pad_to_max_length=True,
                                         return_attention_mask=True,
                                         return_tensors='pt',
                                         )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)


prediction_data = TensorDataset(input_ids, attention_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data,sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
model.eval()

predictions = []

for batch in prediction_dataloader:
    batch = tuple(t.to(device) for t in batch)

    b_input_ids, b_input_mask = batch

    with torch.no_grad():
        outputs = model(b_input_ids,
                        token_type_ids=None, attention_mask=b_input_mask)

    logits = outputs[0]

    logits = logits.detach().cpu().numpy()

    predictions.extend(logits.flatten())



In [ ]:
data['sentiment'] = [predictions]
data.drop('text', axis=1)
res.to_csv(ans_path, index=False)